In [7]:
import numpy as np
from multiview_generator.multiple_sub_problems import MultiViewSubProblemsGenerator

n_views=3
n_classes=3
complementarity = np.array([0.3 for _ in range(n_classes)]).reshape((n_classes, 1))
complementarity_level = np.array([0.5 for _ in range(n_classes)]).reshape((n_classes, 1))
n_examples_per_class = np.array([100 for _ in range(n_classes)])
available_init_indices = [[i+(100*class_ind) for i in range(100)] for class_ind in range(n_classes)]
error_matrix = np.zeros((12,12))
complementarity_examples = [_ for _ in range(n_classes)]
good_views_indices = [_ for _ in range(n_classes)]
bad_views_indices = [_ for _ in range(n_classes)]
rs = np.random.RandomState(42)
example_ids = np.zeros(sum(n_examples_per_class), dtype="S100")

print(available_init_indices)

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99], [100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199], [200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220,

In [8]:
def _remove_available(available_indices, to_remove, class_index):
    """
    Removes indices from the available ones array
    """
    available_indices[class_index] = [ind
                                      for ind
                                      in available_indices[class_index]
                                      if ind not in to_remove]
    return available_indices

def _update_example_indices(target, target_name, class_ind):
    for ind, target_ind in enumerate(target):
        example_ids[target_ind] = target_name + "_{}_{}".format(ind, class_ind)

[[array([0, 2]), array([0, 2]), array([0, 2]), array([0, 1]), array([1, 2]), array([0, 1]), array([0, 2]), array([0, 1]), array([0, 2]), array([1, 2]), array([0, 2]), array([0, 1]), array([0, 2]), array([0, 2]), array([1, 2]), array([0, 2]), array([0, 1]), array([0, 2]), array([0, 1]), array([0, 2]), array([0, 2]), array([1, 2]), array([0, 2]), array([1, 2]), array([0, 2]), array([0, 1]), array([0, 1]), array([0, 2]), array([0, 2]), array([0, 2])], [array([0, 2]), array([1, 2]), array([0, 2]), array([0, 2]), array([0, 1]), array([1, 2]), array([0, 1]), array([1, 2]), array([1, 2]), array([0, 1]), array([0, 1]), array([0, 1]), array([1, 2]), array([0, 2]), array([0, 2]), array([0, 2]), array([0, 2]), array([1, 2]), array([1, 2]), array([0, 1]), array([0, 2]), array([1, 2]), array([0, 1]), array([0, 1]), array([0, 2]), array([0, 1]), array([1, 2]), array([1, 2]), array([1, 2]), array([0, 2])], [array([1, 2]), array([0, 2]), array([0, 1]), array([0, 1]), array([1, 2]), array([0, 1]), arra

In [10]:
n_bad = [int(complementarity_level[class_index]*n_views)
          for class_index in range(n_classes)]
n_bad

[1, 1, 1]

Complementarity is defined by class. Which means that samples of class i can be very complementary for example.
To check if the setting is compatible with the error matrix, 
now we check if there is enough available indices that are not redundant or mutual error.
 

In [12]:
((complementarity * n_examples_per_class)[0] > np.array(
        [len(inds) for inds in available_init_indices]))

array([False, False, False])

In [ ]:
for class_index, complementarity in enumerate(complementarity):
        n_comp = int(complementarity_level[class_index]*n_views)
        complementarity_examples[class_index] = rs.choice(
            available_init_indices[class_index],
            size=int(n_examples_per_class[
                         class_index] * complementarity),
            replace=False)
        _update_example_indices(
            complementarity_examples[class_index],
            'Complementary', class_index)
        good_views_indices[class_index] = [
            rs.choice(np.arange(n_views),
                           size=n_bad,
                           replace=False)
            for _ in complementarity_examples[class_index]]
        bad_views_indices[class_index] = [np.array([ind
                                                         for ind
                                                         in range(
                n_views)
                                                         if ind not in
                                                         good_views_indices[
                                                             class_index][
                                                             ex_ind]])
                                               for ex_ind, _ in
                                               enumerate(
                                                   complementarity_examples[
                                                       class_index])]
        _remove_available(available_init_indices,
                               complementarity_examples[
                                   class_index],
                               class_index)
print(bad_views_indices)                

